## 对话智能体中的提示语结构

1. 确认是否新对话
2. 补充提问
    - 追加到旧的对话
    - 使用提示语构建新对话
    - 结合 init_messages 构建新对话
3. 构建对话
    - 收集最近记忆
    - 结合知识检索
    - 结合工具箱

In [1]:
from illufly.chat import FakeLLM
from illufly.types import PromptTemplate

llm = FakeLLM()
llm("写一首诗词吧", verbose=True, new_chat=True)
llm.memory

ImportError: cannot import name 'BaseEventsHistory' from 'illufly.core.history' (/Users/xuehongwei/codeup/illufly/illufly/core/history/__init__.py)

### 自动生成新的 thread_id

In [2]:
llm = FakeLLM()
llm.build_chat_memory("hi")

[{'role': 'user', 'content': 'hi'}]

In [3]:
llm.thread_id

'715925-9209-0000'

### 手动构建新的 thread_id

In [4]:
llm.create_new_thread()

'715926-1606-0000'

In [5]:
llm.build_chat_memory("hi", new_chat=True)
llm.thread_id

'715926-1606-0000'

## 构建对话

### 新对话

In [3]:
from illufly.chat import FakeLLM
from illufly.types import PromptTemplate

llm = FakeLLM()
llm("写一首诗词吧")
llm.memory

[USER] 写一首诗词吧
Reply >> 写一首诗词吧


[{'role': 'user', 'content': '写一首诗词吧'},
 {'role': 'assistant', 'content': 'Reply >> 写一首诗词吧'}]

### 连续对话

In [4]:
llm("再写一首")
llm.memory

[USER] 再写一首
Reply >> 再写一首


[{'role': 'user', 'content': '写一首诗词吧'},
 {'role': 'assistant', 'content': 'Reply >> 写一首诗词吧'},
 {'role': 'user', 'content': '再写一首'},
 {'role': 'assistant', 'content': 'Reply >> 再写一首'}]

### 如果使用 new_chat 标志

In [5]:
llm("再写一首", new_chat=True)
llm.memory

[USER] 再写一首
Reply >> 再写一首


[{'role': 'user', 'content': '再写一首'},
 {'role': 'assistant', 'content': 'Reply >> 再写一首'}]

### 在调用时使用 system 角色的提示语

In [6]:
llm([("system", "你是一个作家"), ("user", "写一首诗")])
llm.memory

[USER] 写一首诗
Reply >> 写一首诗


[{'role': 'system', 'content': '你是一个作家'},
 {'role': 'user', 'content': '写一首诗'},
 {'role': 'assistant', 'content': 'Reply >> 写一首诗'}]

### 在调用时使用 system 角色并补充 user 消息

In [7]:
llm([("system", "你是一个作家，帮我写首诗")])
llm.memory

[USER] 你是一个作家，帮我写首诗
Reply >> 请开始


[{'role': 'system', 'content': '你是一个作家，帮我写首诗'},
 {'role': 'user', 'content': '请开始'},
 {'role': 'assistant', 'content': 'Reply >> 请开始'}]

In [8]:
llm("再来一首")
llm.memory

[USER] 再来一首
Reply >> 再来一首


[{'role': 'system', 'content': '你是一个作家，帮我写首诗'},
 {'role': 'user', 'content': '请开始'},
 {'role': 'assistant', 'content': 'Reply >> 请开始'},
 {'role': 'user', 'content': '再来一首'},
 {'role': 'assistant', 'content': 'Reply >> 再来一首'}]

### 在声明实例时使用 system 角色的提示语

In [9]:
from illufly.chat import FakeLLM, ChatQwen
from illufly.types import PromptTemplate

llm = ChatQwen(memory=[("system", "你是一个作家")])
llm("写两句7字对联")
llm.memory

[USER] 写两句7字对联
春风化雨润万物，
秋水长天共一色。


[{'role': 'system', 'content': '你是一个作家'},
 {'role': 'user', 'content': '写两句7字对联'},
 {'role': 'assistant', 'content': '春风化雨润万物，\n秋水长天共一色。'}]

In [10]:
llm("再来一首")
llm.memory

[USER] 再来一首
晨光破晓迎新岁，
晚霞映日送旧年。


[{'role': 'system', 'content': '你是一个作家'},
 {'role': 'user', 'content': '写两句7字对联'},
 {'role': 'assistant', 'content': '春风化雨润万物，\n秋水长天共一色。'},
 {'role': 'user', 'content': '再来一首'},
 {'role': 'assistant', 'content': '晨光破晓迎新岁，\n晚霞映日送旧年。'}]

### 进一步简化：仅使用字符串或提示语模板构造 memory
在大多数情况下，使用提示语模板是为了构造 system 消息，而使用单纯字符串是为了构造 user 消息。
因此，我们按照这两种常见情况构建了语法糖的规则：

In [11]:
from illufly.chat import FakeLLM

llm = FakeLLM(memory=["你是一个作家", "请开始"], response="好")
llm("写一首诗词吧", new_chat=True)
llm.memory

[USER] 写一首诗词吧
好


[{'role': 'user', 'content': '你是一个作家'},
 {'role': 'assistant', 'content': '请开始'},
 {'role': 'user', 'content': '写一首诗词吧'},
 {'role': 'assistant', 'content': '好'}]

### 绑定 task 并自动补充 user 消息

In [13]:
llm = FakeLLM(memory=[("system", PromptTemplate(text="你是一个作家，请帮我{{task}}"))])
llm("写一首诗词吧")
llm.memory

[USER] 写一首诗词吧
Reply >> 请开始


[{'role': 'system', 'content': '你是一个作家，请帮我写一首诗词吧'},
 {'role': 'user', 'content': '请开始'},
 {'role': 'assistant', 'content': 'Reply >> 请开始'}]

### 绑定 task 并提供 user 消息

In [15]:
llm = FakeLLM(memory=[
    ("system", PromptTemplate(text="你是一个作家，请帮我{{task}}")), 
    ("user", "开始吧")
])
llm("写一首诗词吧")
llm.memory

[USER] 写一首诗词吧
Reply >> 开始吧


[{'role': 'system', 'content': '你是一个作家，请帮我写一首诗词吧'},
 {'role': 'user', 'content': '开始吧'},
 {'role': 'assistant', 'content': 'Reply >> 开始吧'}]

In [16]:
llm("再来")
llm.memory

[USER] 再来
Reply >> 再来


[{'role': 'system', 'content': '你是一个作家，请帮我写一首诗词吧'},
 {'role': 'user', 'content': '开始吧'},
 {'role': 'assistant', 'content': 'Reply >> 开始吧'},
 {'role': 'user', 'content': '再来'},
 {'role': 'assistant', 'content': 'Reply >> 再来'}]

## 映射提示语中的模板变量

### 默认映射

In [17]:
from illufly.chat import FakeLLM
from illufly.types import PromptTemplate

llm = FakeLLM(
    memory=[PromptTemplate(text="请你帮我写{{task}}"), "请开始"]
)
llm("一首儿歌，4行")
llm.memory

[USER] 一首儿歌，4行
Reply >> 请开始


[{'role': 'system', 'content': '请你帮我写一首儿歌，4行'},
 {'role': 'user', 'content': '请开始'},
 {'role': 'assistant', 'content': 'Reply >> 请开始'}]

### 重新映射

In [18]:
from illufly.chat import FakeLLM
from illufly.types import PromptTemplate

llm = FakeLLM(
    memory=[
        PromptTemplate(text="请你帮我写{{mytask}}", lazy_binding_map={"mytask": "task"}), 
        "请开始"
    ]
)
llm("一首儿歌，4行")
llm.memory

[USER] 一首儿歌，4行
Reply >> 请开始


[{'role': 'system', 'content': '请你帮我写一首儿歌，4行'},
 {'role': 'user', 'content': '请开始'},
 {'role': 'assistant', 'content': 'Reply >> 请开始'}]

In [19]:
from illufly.chat import FakeLLM
from illufly.types import PromptTemplate

llm = FakeLLM(
    memory=[
        PromptTemplate(text="请你帮我写{{mytask}}", lazy_binding_map={"mytask": "task"}), 
        "请开始"
    ]
)
llm([PromptTemplate(text="写一首歌{{task}}"), "一首儿歌，4行"])
llm.memory

[USER] 一首儿歌，4行
Reply >> 请开始


[{'role': 'system', 'content': '写一首歌一首儿歌，4行'},
 {'role': 'user', 'content': '请开始'},
 {'role': 'assistant', 'content': 'Reply >> 请开始'}]

## 在记忆中携带背景知识

实现RAG应用的基本原理就是在记忆中嵌入检索到的资料，以下是一个简单的示范。<br>
这里是为了讲解「记忆模块」，实现RAG应用本身这个复杂话题请参考专门的RAG专题。

### 嵌入到提示语中

In [27]:
from illufly.chat import FakeLLM
from illufly.types import PromptTemplate

llm = FakeLLM(
    memory=PromptTemplate(text="已知：{{knowledge}}"),
    knowledge=["illufly 是一个多智能体解决方案"]
)

llm("illufly 是什么？")
llm.memory

[USER] illufly 是什么？
Reply >> illufly 是什么？


[{'role': 'system', 'content': "已知：['illufly 是一个多智能体解决方案']"},
 {'role': 'user', 'content': 'illufly 是什么？'},
 {'role': 'assistant', 'content': 'Reply >> illufly 是什么？'}]

In [24]:
llm.provider_dict

{'last_output': 'Reply >> 写一首诗词吧',
 'resources': '',
 'task': '写一首诗词吧',
 'final_answer': '',
 'tools_calling_steps': [],
 'tools_name': '',
 'tools_desc': '',
 'knowledge': ['illufly 是一个多智能体解决方案'],
 'recalled_knowledge': ['illufly 是一个多智能体解决方案']}

### 补充到对话记录中

将知识嵌入到提示语模板也许是流行的思路，但自然而然的做法是什么都不做，由 ChatAgent 自己补充到对话记录中。<br>
这样设计并不显得突兀：既然都已经指定了知识，有什么理由不使用呢？

In [28]:
from illufly.chat import FakeLLM
from illufly.types import PromptTemplate

llm = FakeLLM(
    memory=PromptTemplate(text="你是一个咨询助手"),
    knowledge=["illufly 是一个多智能体解决方案"]
)

llm("illufly 是什么？")
llm.memory

[USER] illufly 是什么？
Reply >> illufly 是什么？


[{'role': 'system', 'content': '你是一个咨询助手'},
 {'role': 'user', 'content': '回答时请参考已有知识：\n@knowledge\nillufly 是一个多智能体解决方案\n'},
 {'role': 'assistant', 'content': 'ok'},
 {'role': 'user', 'content': 'illufly 是什么？'},
 {'role': 'assistant', 'content': 'Reply >> illufly 是什么？'}]

### ChatQwen 示范

In [29]:
from illufly.chat import ChatQwen
from illufly.types import PromptTemplate

llm = ChatQwen(knowledge=["illufly 是一个多智能体解决方案"])
llm("illufly 是什么？")
llm.memory

[USER] illufly 是什么？
illufly 是一个多智能体解决方案，它通过集成多个智能体来协同完成复杂任务或提供综合服务。这种解决方案通常应用于需要高度智能化和自动化处理的场景，比如智能交通系统、智慧城市管理、自动化生产调度等。通过多智能体之间的协作与通信，illufly 能够实现更高效、更灵活的问题解决方式。


[{'role': 'user', 'content': '回答时请参考已有知识：\n@knowledge\nillufly 是一个多智能体解决方案\n'},
 {'role': 'assistant', 'content': 'ok'},
 {'role': 'user', 'content': 'illufly 是什么？'},
 {'role': 'assistant',
  'content': 'illufly 是一个多智能体解决方案，它通过集成多个智能体来协同完成复杂任务或提供综合服务。这种解决方案通常应用于需要高度智能化和自动化处理的场景，比如智能交通系统、智慧城市管理、自动化生产调度等。通过多智能体之间的协作与通信，illufly 能够实现更高效、更灵活的问题解决方式。'}]

## 管理记忆长度

### 默认记住10轮

In [1]:
from illufly.chat import FakeLLM
from illufly.types import PromptTemplate

a = FakeLLM()

a("你能帮我写一首关于兔子做梦的四句儿歌?", verbose=True)
a.chat_memory

[USER] 你能帮我写一首关于兔子做梦的四句儿歌?
[INFO] 记住 10 轮对话
history_memory []
[INFO] I am FakeLLM
Reply >> 你能帮我写一首关于兔子做梦的四句儿歌?


[{'role': 'user', 'content': '你能帮我写一首关于兔子做梦的四句儿歌?'},
 {'role': 'assistant', 'content': 'Reply >> 你能帮我写一首关于兔子做梦的四句儿歌?'}]

### 限制记忆中的对话轮数

In [2]:
a("改为在蓝天上", verbose=True)
a.chat_memory

[USER] 改为在蓝天上
[INFO] 记住 10 轮对话
history_memory [Message(role=user, content=你能帮我写一首关于兔子做梦的四句儿歌?), Message(role=assistant, content=Reply >> 你能帮我写一首关于兔子做梦的四句儿歌?)]
[INFO] I am FakeLLM
Reply >> 改为在蓝天上


[{'role': 'user', 'content': '你能帮我写一首关于兔子做梦的四句儿歌?'},
 {'role': 'assistant', 'content': 'Reply >> 你能帮我写一首关于兔子做梦的四句儿歌?'},
 {'role': 'user', 'content': '改为在蓝天上'},
 {'role': 'assistant', 'content': 'Reply >> 改为在蓝天上'}]

In [3]:
a("改为在水塘", remember_rounds=1, verbose=True)
a.chat_memory

[USER] 改为在水塘
[INFO] 记住 1 轮对话
history_memory [Message(role=user, content=改为在蓝天上), Message(role=assistant, content=Reply >> 改为在蓝天上)]
[INFO] I am FakeLLM
Reply >> 改为在水塘


[{'role': 'user', 'content': '改为在蓝天上'},
 {'role': 'assistant', 'content': 'Reply >> 改为在蓝天上'},
 {'role': 'user', 'content': '改为在水塘'},
 {'role': 'assistant', 'content': 'Reply >> 改为在水塘'}]